In [1]:
from browser_use import Agent, ChatGoogle
from dotenv import load_dotenv
import os
from datetime import datetime
import asyncio
import re
import json

# Load GOOGLE_API_KEY from .env
load_dotenv()

# Initialize the Gemini model
llm = ChatGoogle(model="gemini-2.5-flash")

# Create the browsing agent with browser configuration
agent = Agent(
    task=(
        "Find upcoming events in the next two weeks that could impact the US stock market. "
        "Look for Federal Reserve meetings, earnings announcements from major companies, "
        "economic data releases, and other market-moving events. "
        "Summarize with date, event type, and why it matters for investors."
    ),
    llm=llm,
    browser_config={
        "headless": True,
        "browser_type": "chromium",
        "browser_timeout": 60,
        "viewport_size": {"width": 1280, "height": 720},
        "extra_chromium_args": [
            "--no-sandbox",
            "--disable-dev-shm-usage",
            "--disable-gpu",
            "--disable-extensions",
            "--disable-plugins",
            "--disable-images",
            "--disable-javascript",
        ]
    }
)

# ---- Helper functions for structuring ----
def extract_events_from_text(text: str):
    """
    Extract structured events (date, event, type, reason) from raw agent output text.
    Very simple regex/NLP pass – can be upgraded with LLM parsing.
    """
    events = []

    # Regex for dates like "Sept 18, 2025" or "September 10"
    date_pattern = r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Sept|Oct|Nov|Dec)[a-z]*\.?\s+\d{1,2}(?:,\s*\d{4})?)'

    lines = text.splitlines()
    for line in lines:
        date_match = re.search(date_pattern, line, flags=re.IGNORECASE)
        if date_match:
            date = date_match.group(1).strip()
            # Try to infer event type
            if "Fed" in line or "FOMC" in line:
                etype = "Federal Reserve"
            elif "earnings" in line or "report" in line:
                etype = "Earnings"
            elif "CPI" in line or "inflation" in line:
                etype = "Economic Data"
            elif "jobs" in line or "employment" in line:
                etype = "Economic Data"
            else:
                etype = "Other"

            event = {
                "date": date,
                "event": line.strip(),
                "type": etype,
                "impact_reason": "Market-moving event (auto-extracted)"
            }
            events.append(event)

    return events


async def main():
    try:
        print("🚀 Starting market events analysis...")
        history = await asyncio.wait_for(agent.run(max_steps=25), timeout=300)

        md_path = "market_events_report.md"
        summary_path = "market_events_summary.txt"
        structured_path = "market_events_structured.json"

        # ---- Save final result into reports ----
        final_result = history.final_result() or ""

        with open(md_path, "w", encoding="utf-8") as f:
            f.write("# Market Events Agent Report\n\n")
            f.write(f"*Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*\n\n")
            f.write("## Final Result\n")
            f.write(final_result + "\n")

        with open(summary_path, "w", encoding="utf-8") as f:
            f.write("MARKET EVENTS SUMMARY\n")
            f.write("=" * 50 + "\n\n")
            f.write(final_result if final_result else "No results were generated by the agent.")

        print(f"✅ Reports saved: {md_path}, {summary_path}")

        # ---- Step 24–37: Structuring Events ----
        structured_events = extract_events_from_text(final_result)

        with open(structured_path, "w", encoding="utf-8") as f:
            json.dump(structured_events, f, indent=2)

        print(f"📦 Structured JSON saved to {structured_path}")
        print(json.dumps(structured_events, indent=2))

    except Exception as e:
        print(f"❌ Error occurred: {str(e)}")

# For Jupyter / Async environments
await main()


INFO     [service] Using anonymized telemetry, see https://docs.browser-use.com/development/telemetry.
🚀 Starting market events analysis...
INFO     [Agent] 🚀 Task: Find upcoming events in the next two weeks that could impact the US stock market. Look for Federal Reserve meetings, earnings announcements from major companies, economic data releases, and other market-moving events. Summarize with date, event type, and why it matters for investors.
INFO     [Agent] 🧠 Starting a browser-use version 0.7.3 with model=gemini-2.5-flash
INFO     [Agent] 

INFO     [Agent] 📍 Step 1:
INFO     [Agent]   👍 Eval: The previous goal was to start the task, and the current state is an empty browser and an empty todo.md. Verdict: Success.
INFO     [Agent]   🎯 Next goal: Create a detailed plan in `todo.md` to outline the steps for finding and summarizing market-moving events.
INFO     [Agent]   🦾 [ACTION 1/1] write_file: file_name: todo.md, content: # US Stock Market Impacting Events - Next Two Weeks (202

In [ ]:
from browser_use import Agent, ChatGoogle
from dotenv import load_dotenv
import os
import asyncio
from datetime import datetime
import json

# ===============================
# Setup
# ===============================

load_dotenv()
llm = ChatGoogle(model="gemini-2.5-flash")

sources = [
    "investopedia.com",
    "marketwatch.com",
    "seekingalpha.com",
    "fool.com",
    "finance.yahoo.com",
    "zacks.com",
    "morningstar.com",
    "investorplace.com",
    "investing.com",
    "barrons.com",
    "cfainstitute.org",
    "ssrn.com",
    "nber.org",
    "aqr.com",
    "researchaffiliates.com"
]

visited_sources_file = "visited_sources.json"
markdown_path = "momentum_investing_summary.md"
structured_path = "momentum_investing_data.json"

# ===============================
# Helpers
# ===============================

def load_visited_sources():
    if os.path.exists(visited_sources_file):
        with open(visited_sources_file, "r", encoding="utf-8") as f:
            return set(json.load(f))
    return set()

def save_visited_sources(visited):
    with open(visited_sources_file, "w", encoding="utf-8") as f:
        json.dump(list(visited), f, indent=2)

def get_remaining_sources():
    visited = load_visited_sources()
    return [s for s in sources if s not in visited]

def append_markdown(new_text, path=markdown_path):
    with open(path, "a", encoding="utf-8") as f:
        f.write("\n\n---\n\n")
        f.write(new_text)

def append_structured(new_data, path=structured_path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf-8") as f:
            existing = json.load(f)
    else:
        existing = []

    if isinstance(new_data, str):
        try:
            new_data = json.loads(new_data)
        except:
            new_data = [{"raw": new_data}]

    combined = existing + new_data
    with open(path, "w", encoding="utf-8") as f:
        json.dump(combined, f, indent=2)

# ===============================
# Agent Setup
# ===============================

def make_agent(remaining_sources):
    return Agent(
        task=(
            "Search for the latest investor research and commentary on momentum investing, "
            "focusing on how strategies adapt to changing market dynamics. "
            "Summarize key takeaways, including insights on portfolio construction, "
            "trading costs, holding periods, scalability, and adaptation. "
            "Return a clean markdown summary with sections: Key Findings, Implications for Investors, "
            "and Practical Recommendations. "
            f"Restrict yourself to these sources: {', '.join(remaining_sources)}."
        ),
        llm=llm,
        browser_config={
            "headless": True,
            "browser_type": "chromium",
            "browser_timeout": 60,
            "viewport_size": {"width": 1280, "height": 720},
            "extra_chromium_args": [
                "--no-sandbox",
                "--disable-dev-shm-usage",
                "--disable-gpu",
                "--disable-extensions",
                "--disable-plugins",
                "--disable-images",
                "--disable-javascript",
            ]
        }
    )

# ===============================
# Main Run (manual trigger)
# ===============================

async def run_research(batch_size=3):
    visited = load_visited_sources()
    remaining = get_remaining_sources()

    if not remaining:
        print("✅ All sources already covered.")
        return

    next_batch = remaining[:batch_size]
    print(f"🔍 Running research on: {', '.join(next_batch)}")

    agent = make_agent(next_batch)

    try:
        history = await asyncio.wait_for(agent.run(max_steps=25), timeout=300)
    except asyncio.TimeoutError:
        print("❌ Agent timed out")
        return

    final_result = history.final_result()
    if final_result:
        # Save markdown
        append_markdown(f"## Research Batch ({', '.join(next_batch)})\n\n{final_result}")

        # Save structured if available
        if hasattr(history, "structured_output") and history.structured_output:
            append_structured(history.structured_output)

        print(f"✅ Results from {next_batch} saved.")
    else:
        print("⚠️ No final result produced.")

    # Update visited
    visited.update(next_batch)
    save_visited_sources(visited)

# ===============================
# Manual Execution
# ===============================

# Example: run one batch
if __name__ == "__main__":
    print("🔧 Manual research agent ready.")
    print("➡️ Run: await run_research() inside Jupyter or asyncio.run(run_research()) in script")
